<a href="https://colab.research.google.com/github/sailubandi/Person-Detection-Using-DETR/blob/main/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers timm opencv-python numpy pandas pytorch-video


ERROR: Could not find a version that satisfies the requirement pytorch-video (from versions: none)
ERROR: No matching distribution found for pytorch-video


In [1]:
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
  print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')

input_video_path = filename # Store the uploaded filename


Saving pinterestdownloader.com-1749549530.251401.mp4 to pinterestdownloader.com-1749549530.251401.mp4
User uploaded file "pinterestdownloader.com-1749549530.251401.mp4" with length 3032221 bytes


In [2]:
import cv2
import os
# Import the correct classes for DETR
from transformers import DetrFeatureExtractor, DetrForObjectDetection
import torch
from PIL import Image
import numpy as np
from google.colab.patches import cv2_imshow  # For displaying images in Colab

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pre-trained model and feature extractor
model_name = "facebook/detr-resnet-50"  # Or try "facebook/detr-resnet-50-dc5" for smaller model
# Load the correct model class
model = DetrForObjectDetection.from_pretrained(model_name).to(device)
# Load the correct feature extractor class
from transformers import AutoImageProcessor
feature_extractor = AutoImageProcessor.from_pretrained(model_name)

model.eval()

def detect_persons(frame_path, model, feature_extractor, threshold=0.8):
    image = Image.open(frame_path)
    inputs = feature_extractor(images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    # Convert outputs to COCO API format
    target_sizes = torch.tensor([image.size[::-1]]).to(device)
    results = feature_extractor.post_process_object_detection(
        outputs, threshold=threshold, target_sizes=target_sizes
    )[0]

    person_boxes = []
    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        # Ensure the label mapping is correct for the loaded model
        if model.config.id2label[label.item()] == "person" and score.item() > threshold:
            box = [int(i) for i in box.tolist()]
            person_boxes.append(box)

    return person_boxes, len(person_boxes)

def visualize_detections(frame, boxes, count, frame_number=None):
    """
    Draw bounding boxes and count information on the frame
    """
    # Draw each bounding box
    for box in boxes:
        xmin, ymin, xmax, ymax = box
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

    # Create information panel
    panel_height = 80
    panel = np.zeros((panel_height, frame.shape[1], 3), dtype=np.uint8)

    # Display count information
    cv2.putText(panel, f"Persons detected: {count}", (20, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    if frame_number is not None:
        cv2.putText(panel, f"Frame: {frame_number}", (20, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Combine frame with panel
    visualized_frame = np.vstack([panel, frame])

    return visualized_frame

def process_video(video_path, output_path, frame_interval=10):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Adjust height for the information panel
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height + 80))

    frame_count = 0
    person_counts = []
    # Initialize boxes and count before the loop to avoid NameError in visualize_detections
    # if the first frames are skipped by frame_interval
    boxes = []
    count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Only run detection at the specified interval
        if frame_count % frame_interval == 0:
            # Save temporary frame
            temp_path = f"temp_frame_{frame_count}.jpg"
            cv2.imwrite(temp_path, frame)

            # Detect persons
            boxes, count = detect_persons(temp_path, model, feature_extractor)
            person_counts.append(count)
            os.remove(temp_path)

        # Visualize detections for every frame, using the last calculated boxes and count
        visualized_frame = visualize_detections(frame.copy(), boxes, count, frame_count)

        out.write(visualized_frame)
        frame_count += 1

    cap.release()
    out.release()

    return person_counts

if __name__ == "__main__":
    input_video = input_video_path  # Use the uploaded video path
    output_video = "output.mp4"
    counts = process_video(input_video, output_video)
    print(f"Person counts per frame interval: {counts}")
    print(f"Video saved to {output_video}")

Using device: cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.59k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for conv1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for bn1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for bn1.bias: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pas

preprocessor_config.json:   0%|          | 0.00/290 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Person counts per frame interval: [6, 5, 7, 6, 7, 5, 5, 6, 6, 6, 5, 6, 7, 7, 8, 9, 8, 6, 6, 7, 7, 5, 4, 5, 5, 4, 5, 4, 2, 2, 2, 3, 3, 3, 3, 3, 3]
Video saved to output.mp4


In [3]:
from google.colab import files
files.download("output.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>